In [1]:
def read_file(file):
    import csv
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        header = next(reader)
        contacts = [[line[0], line[1]] for line in reader]
    return contacts

In [2]:
def read_template(message):
    from string import Template
    with open(message, 'r', encoding='utf-8') as f:
        content = f.read()
    return Template(content)

In [38]:
def send_messages(contact_file, message_file):
    
    recipients = read_file(contact_file)
    message_template = read_template(message_file)
    
    import smtplib
    from mail_data import EMAIL, PASS
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(EMAIL, PASS)
    
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email.mime.image import MIMEImage
    from email.header import Header
    import html
    import cgi
    import uuid
    
#     from email import encoders
#     from PIL import Image
#     from io import BytesIO

    for contact in recipients:
        name = contact[0]
        email = contact[1]

        message = MIMEMultipart()

        named_template = message_template.substitute(RECIPIENT_NAME=name.title())

        message['From'] = EMAIL
        message['To'] = email
        message['Subject'] = "Subject"
        message.attach(MIMEText(named_template, 'plain'))
        
        # add embeded image:
        
        img = dict(title='Motivation image', path='cleaning.jpeg', cid=str(uuid.uuid4()))
#         msg_text = MIMEText('[image: {title}]'.format(**img), 'plain', 'utf-8')
#         message.attach(msg_text)
        
        msg_html = MIMEText('<br><br><div dir="ltr">'
                     '<img src="cid:{cid}" alt="{alt}"><br></div>'
                    .format(alt=cgi.html.escape(img['title'], quote=True), **img),
                    'html', 'utf-8')
        
        message.attach(msg_html)

        with open(img['path'], 'rb') as file:
#             msg_image = MIMEImage(file.read(), name=os.path.basename(img['path']))
            msg_image = MIMEImage(file.read())
#             msg_image.add_header('Content-ID', f'<{img["cid"]}>')
            message.attach(msg_image)
        
        
#         Add image as attachment:
#         image = Image.open('img.jpeg')
#         bytes = BytesIO()
#         image.save(bytes, format='JPEG')
#         att = MIMEBase('image', 'jpeg')
#         att.set_payload(bytes.getvalue())
#         encoders.encode_base64(att)
#         att.add_header('Content-Disposition', 'attachment', filename='img.jpeg')
     
#         message.attach(att)
        
#         print(message)
        
        server.send_message(message)

        del message
    
    server.quit()


In [39]:
send_messages('contacts_example.csv', 'message_example.txt')

** Create a DB and insert dummy users **

In [17]:
import pymongo
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.email_app
collection=db.users

In [18]:
users = [{'username': 'admin', 'password': 'password'}, {'username': 'guest', 'password': '321321'}]
for user in users:
    collection.insert_one(user)

In [43]:
a = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]

In [52]:
a[:-2:-6]

[9]

In [36]:
import cgi
import uuid
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text      import MIMEText
from email.mime.image     import MIMEImage
from email.header         import Header 
from mail_data import EMAIL, PASS
import html

name = 'pi pu'
email = 'someemail'

message_template = read_template('message_example.txt')
named_template = message_template.substitute(RECIPIENT_NAME=name.title())

# img = dict(title='Motivation report', path='cleaning.jpeg', cid=str(uuid.uuid4()))
image = 'cleaning.jpeg'

msg = MIMEMultipart()
msg['Subject'] = Header('Report', 'utf-8')
msg['From'] = EMAIL
msg['To'] = email
# msg_alternative = MIMEMultipart('alternative')
# msg.attach(msg_alternative)
# msg_alternative.attach(MIMEText(named_template, 'plain'))
msg.attach(MIMEText(named_template, 'plain'))
# msg_text = MIMEText('[image: {title}]'.format(**img), 'plain', 'utf-8')
# msg_alternative.attach(msg_text)
# msg.attach(msg_text)

msgText = MIMEText('<br><div><br><img src="cid:%s"></div><br>' % (image), 'html')  
msg.attach(msgText)

# msg_html = MIMEText('<br><br><div>'
#                      '<img src="cid:{cid}" alt="{alt}"><br></div>'
#                     .format(alt=html.escape(img['title'], quote=True), **img),
#                     'html', 'utf-8')
# msg_alternative.attach(msg_html)
# msg.attach(msg_html)

# fp = open(image, 'rb')                                                    
# img = MIMEImage(fp.read())
# fp.close()
# # img.add_header('Content-ID', '<div>Image</div>')
# msg.attach(img)

with open(image, 'rb') as file:
    msg_image = MIMEImage(file.read(), name=os.path.basename(image))
#     msg_image.add_header('Content-ID', f'<{img["cid"]}>')
    msg.attach(msg_image)

In [31]:
msg

In [21]:
import smtplib
from mail_data import EMAIL, PASS
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(EMAIL, PASS)

(235, b'2.7.0 Accepted')

In [37]:
server.send_message(msg)

{}